In [102]:
using System;
using System.Threading;

class DefiniteIntegral
{
    public static double Solve(double a, double b, Func<double, double> function, double step, int threadsNumber)
    {
        double result = 0.0;
        double intervalWidth = (b - a) / threadsNumber;
        ManualResetEvent[] doneThread = new ManualResetEvent[threadsNumber];

        for (int i = 0; i < threadsNumber; i++)
        {
            doneThread[i] = new ManualResetEvent(false);
            int threadIndex = i;
            ThreadPool.QueueUserWorkItem((state) =>
            {
                double startPoint = a + threadIndex * intervalWidth;
                double endPoint = startPoint + intervalWidth;
                double sum = (function(startPoint) + function(endPoint)) / 2.0;

                for (double x = startPoint + step; x < endPoint; x += step)
                {
                    sum += function(x);
                }

                sum *= step;
                result += sum;
                doneThread[threadIndex].Set(); 
            });
        }

        WaitHandle.WaitAll(doneThread); 

        return result;
    }
}

In [104]:
#r "nuget: xunit, 2.8.1"

using Xunit;

var X = (double x) => x;
var SIN = (double x) => Math.Sin(x);

Assert.Equal(0, DefiniteIntegral.Solve(-1, 1, X, 1e-4, 2), 1e-4);

Assert.Equal(0, DefiniteIntegral.Solve(-1, 1, SIN, 1e-5, 8), 1e-4);

Assert.Equal(50, DefiniteIntegral.Solve(0, 10, X, 1e-6, 8), 1e-5);

Installed Packages xunit, 2.8.1

Error: Xunit.Sdk.EqualException: Assert.Equal() Failure: Values are not within tolerance 1,0000000000000001E-05
Expected: 50
Actual:   50,000019999674024
   at Xunit.Assert.Equal(Double expected, Double actual, Double tolerance) in /_/src/xunit.assert/Asserts/EqualityAsserts.cs:line 352
   at Submission#105.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)